# Comparativo de Precipitação - ONS vs TOK

Este notebook compara dados de precipitação de duas fontes (ONS e TOK) por pontos de interesse.

## Parâmetros Configuráveis:
- **nome_modelo**: Nome do modelo a ser analisado
- **horizonte**: Horizonte de previsão do modelo
- **data_rodada**: Data da rodada do modelo (formato: YYYYMMDD)
- **diretorio_base**: Diretório base onde os dados estão armazenados

## Saídas:
- Tabelas com heatmap de diferenças
- Acumulado de diferenças positivas e negativas
- Desvio padrão das diferenças

In [ ]:
# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('default')
sns.set_palette('husl')
%matplotlib inline

## 1. Configuração dos Parâmetros

Defina os parâmetros do modelo e caminhos de entrada/saída.

In [ ]:
# ============================================
# PARÂMETROS CONFIGURÁVEIS
# ============================================

# Nome do modelo
nome_modelo = "MODELO_EXEMPLO"

# Horizonte de previsão (em dias ou horas, dependendo do modelo)
horizonte = 7

# Data da rodada (formato: YYYYMMDD)
data_rodada = "20260123"

# Diretório base onde os dados estão armazenados
diretorio_base = "dados_exemplo"  # Use "dados_exemplo" para dados de teste ou especifique seu caminho

# ============================================
# CONSTRUÇÃO DOS CAMINHOS
# ============================================

# Caminho de entrada (varia de acordo com nome do modelo e data da rodada)
caminho_input = Path(diretorio_base) / nome_modelo / data_rodada

# Caminhos específicos para ONS e TOK
caminho_ons = caminho_input / "ONS"
caminho_tok = caminho_input / "TOK"

# Caminho de saída (organizado como o input)
caminho_output = caminho_input / "comparacao"
caminho_output.mkdir(parents=True, exist_ok=True)

print(f"Modelo: {nome_modelo}")
print(f"Horizonte: {horizonte}")
print(f"Data da rodada: {data_rodada}")
print(f"\nCaminhos:")
print(f"  Input: {caminho_input}")
print(f"  ONS: {caminho_ons}")
print(f"  TOK: {caminho_tok}")
print(f"  Output: {caminho_output}")

## 2. Função para Carregar Dados

Funções auxiliares para carregar dados de diferentes fontes.

In [ ]:
def carregar_dados_fonte(caminho, fonte):
    """
    Carrega dados de precipitação de uma fonte específica.
    
    Parâmetros:
    -----------
    caminho : Path
        Caminho para o diretório com os dados
    fonte : str
        Nome da fonte (ONS ou TOK)
    
    Retorna:
    --------
    DataFrame com os dados de precipitação
    """
    # Procura por arquivos CSV, Excel ou Parquet no diretório
    arquivos_csv = list(caminho.glob("*.csv"))
    arquivos_excel = list(caminho.glob("*.xlsx")) + list(caminho.glob("*.xls"))
    arquivos_parquet = list(caminho.glob("*.parquet"))
    
    if arquivos_csv:
        print(f"Carregando dados {fonte} de: {arquivos_csv[0].name}")
        return pd.read_csv(arquivos_csv[0])
    elif arquivos_excel:
        print(f"Carregando dados {fonte} de: {arquivos_excel[0].name}")
        return pd.read_excel(arquivos_excel[0])
    elif arquivos_parquet:
        print(f"Carregando dados {fonte} de: {arquivos_parquet[0].name}")
        return pd.read_parquet(arquivos_parquet[0])
    else:
        print(f"AVISO: Nenhum arquivo de dados encontrado em {caminho}")
        # Retorna dados de exemplo para demonstração
        return gerar_dados_exemplo(fonte)

def gerar_dados_exemplo(fonte):
    """
    Gera dados de exemplo para demonstração quando os arquivos não existem.
    """
    np.random.seed(42 if fonte == "ONS" else 43)
    
    # Cria dados de exemplo com pontos de interesse e séries temporais
    pontos = [f"Ponto_{i:02d}" for i in range(1, 11)]
    datas = pd.date_range(start='2026-01-01', periods=horizonte, freq='D')
    
    data = []
    for ponto in pontos:
        for data_medicao in datas:
            precipitacao = np.random.gamma(2, 10) if fonte == "ONS" else np.random.gamma(2.2, 9.5)
            data.append({
                'ponto': ponto,
                'data': data_medicao,
                'precipitacao_mm': precipitacao
            })
    
    df = pd.DataFrame(data)
    print(f"  (Usando dados de exemplo para {fonte})")
    return df

## 3. Carregar Dados das Duas Fontes

In [ ]:
# Carregar dados ONS
print("Carregando dados ONS...")
dados_ons = carregar_dados_fonte(caminho_ons, "ONS")

# Carregar dados TOK
print("\nCarregando dados TOK...")
dados_tok = carregar_dados_fonte(caminho_tok, "TOK")

# Exibir informações básicas
print(f"\n{'='*60}")
print(f"Dados ONS: {dados_ons.shape[0]} registros")
print(f"Dados TOK: {dados_tok.shape[0]} registros")
print(f"{'='*60}")

# Exibir primeiras linhas
print("\nPrimeiras linhas - ONS:")
display(dados_ons.head())

print("\nPrimeiras linhas - TOK:")
display(dados_tok.head())

## 4. Preparar Dados para Comparação

In [ ]:
# Padronizar nomes de colunas (assumindo que podem ter nomes diferentes)
# Ajuste conforme o formato real dos seus dados
def padronizar_dataframe(df):
    """
    Padroniza o DataFrame com colunas: ponto, data, precipitacao_mm
    """
    # Cria uma cópia para não modificar o original
    df_padrao = df.copy()
    
    # Identifica colunas relevantes (ajuste conforme necessário)
    # Esta é uma implementação genérica que pode precisar de ajustes
    colunas = df_padrao.columns.str.lower()
    
    # Renomeia colunas se necessário
    rename_map = {}
    for col in df_padrao.columns:
        col_lower = col.lower()
        if 'ponto' in col_lower or 'local' in col_lower or 'estacao' in col_lower:
            rename_map[col] = 'ponto'
        elif 'data' in col_lower or 'date' in col_lower:
            rename_map[col] = 'data'
        elif 'precip' in col_lower or 'chuva' in col_lower or 'mm' in col_lower:
            rename_map[col] = 'precipitacao_mm'
    
    if rename_map:
        df_padrao = df_padrao.rename(columns=rename_map)
    
    # Garante que a coluna data está no formato datetime
    if 'data' in df_padrao.columns:
        df_padrao['data'] = pd.to_datetime(df_padrao['data'])
    
    return df_padrao

# Padronizar ambos os DataFrames
dados_ons = padronizar_dataframe(dados_ons)
dados_tok = padronizar_dataframe(dados_tok)

# Merge dos dados para comparação
dados_comparacao = pd.merge(
    dados_ons,
    dados_tok,
    on=['ponto', 'data'],
    suffixes=('_ons', '_tok'),
    how='outer'
)

# Preenche valores ausentes com 0 (ou outra estratégia conforme necessário)
dados_comparacao = dados_comparacao.fillna(0)

print(f"Dados combinados: {dados_comparacao.shape[0]} registros")
display(dados_comparacao.head(10))

## 5. Cálculo das Diferenças

In [ ]:
# Calcular diferenças (ONS - TOK)
dados_comparacao['diferenca'] = dados_comparacao['precipitacao_mm_ons'] - dados_comparacao['precipitacao_mm_tok']

# Calcular diferenças absolutas
dados_comparacao['diferenca_abs'] = dados_comparacao['diferenca'].abs()

# Calcular diferenças percentuais
# Evita divisão por zero
dados_comparacao['diferenca_pct'] = np.where(
    dados_comparacao['precipitacao_mm_tok'] != 0,
    (dados_comparacao['diferenca'] / dados_comparacao['precipitacao_mm_tok']) * 100,
    0
)

# Classificar diferenças
dados_comparacao['tipo_diferenca'] = dados_comparacao['diferenca'].apply(
    lambda x: 'Positiva' if x > 0 else ('Negativa' if x < 0 else 'Sem diferença')
)

print("Estatísticas das diferenças:")
print(f"{'='*60}")
print(dados_comparacao['diferenca'].describe())
print(f"\nDesvio padrão das diferenças: {dados_comparacao['diferenca'].std():.2f} mm")

display(dados_comparacao.head(10))

## 6. Estatísticas por Ponto de Interesse

In [ ]:
# Agregar estatísticas por ponto
stats_por_ponto = dados_comparacao.groupby('ponto').agg({
    'precipitacao_mm_ons': ['mean', 'sum'],
    'precipitacao_mm_tok': ['mean', 'sum'],
    'diferenca': ['mean', 'sum', 'std', 'min', 'max'],
    'diferenca_abs': ['mean', 'max']
}).round(2)

# Renomear colunas para facilitar leitura
stats_por_ponto.columns = [
    'ONS_media', 'ONS_total',
    'TOK_media', 'TOK_total',
    'Dif_media', 'Dif_total', 'Dif_desvio_padrao', 'Dif_min', 'Dif_max',
    'Dif_abs_media', 'Dif_abs_max'
]

print("\nEstatísticas por Ponto de Interesse:")
print(f"{'='*80}")
display(stats_por_ponto)

# Salvar estatísticas
arquivo_stats = caminho_output / f"estatisticas_por_ponto_{data_rodada}.csv"
stats_por_ponto.to_csv(arquivo_stats)
print(f"\nEstatísticas salvas em: {arquivo_stats}")

## 7. Acumulado de Diferenças Positivas e Negativas

In [ ]:
# Separar diferenças positivas e negativas
dif_positivas = dados_comparacao[dados_comparacao['diferenca'] > 0].copy()
dif_negativas = dados_comparacao[dados_comparacao['diferenca'] < 0].copy()

# Calcular acumulados por ponto
acumulado_positivo = dif_positivas.groupby('ponto')['diferenca'].sum().round(2)
acumulado_negativo = dif_negativas.groupby('ponto')['diferenca'].sum().round(2)

# Criar DataFrame consolidado
acumulados = pd.DataFrame({
    'Acumulado_Positivo_mm': acumulado_positivo,
    'Acumulado_Negativo_mm': acumulado_negativo,
    'Total_Liquido_mm': acumulado_positivo + acumulado_negativo
}).fillna(0).round(2)

print("\nAcumulado de Diferenças por Ponto:")
print(f"{'='*60}")
display(acumulados)

# Estatísticas gerais
print(f"\nTOTAL GERAL:")
print(f"  Acumulado Positivo: {acumulado_positivo.sum():.2f} mm")
print(f"  Acumulado Negativo: {acumulado_negativo.sum():.2f} mm")
print(f"  Total Líquido: {acumulados['Total_Liquido_mm'].sum():.2f} mm")

# Salvar acumulados
arquivo_acumulados = caminho_output / f"acumulados_diferencas_{data_rodada}.csv"
acumulados.to_csv(arquivo_acumulados)
print(f"\nAcumulados salvos em: {arquivo_acumulados}")

## 8. Tabela com Desvio Padrão das Diferenças

In [ ]:
# Calcular desvio padrão por ponto e por data
desvio_por_ponto = dados_comparacao.groupby('ponto')['diferenca'].std().round(2)
desvio_por_data = dados_comparacao.groupby('data')['diferenca'].std().round(2)

# Criar tabela de desvios
tabela_desvios = pd.DataFrame({
    'Desvio_Padrao_mm': desvio_por_ponto
})

print("\nDesvio Padrão das Diferenças por Ponto:")
print(f"{'='*60}")
display(tabela_desvios)

print(f"\nDesvio Padrão Geral: {dados_comparacao['diferenca'].std():.2f} mm")

# Salvar desvios
arquivo_desvios = caminho_output / f"desvio_padrao_diferencas_{data_rodada}.csv"
tabela_desvios.to_csv(arquivo_desvios)
print(f"\nDesvios salvos em: {arquivo_desvios}")

## 9. Heatmap de Diferenças por Ponto e Data

In [ ]:
# Criar matriz para heatmap (pontos x datas)
matriz_diferencas = dados_comparacao.pivot_table(
    index='ponto',
    columns='data',
    values='diferenca',
    aggfunc='mean'
).fillna(0)

# Configurar figura
fig, ax = plt.subplots(figsize=(14, 8))

# Criar heatmap
sns.heatmap(
    matriz_diferencas,
    cmap='RdBu_r',
    center=0,
    annot=True,
    fmt='.1f',
    linewidths=0.5,
    cbar_kws={'label': 'Diferença (mm)'},
    ax=ax
)

# Formatação
ax.set_title(f'Heatmap de Diferenças (ONS - TOK)\n{nome_modelo} - {data_rodada}', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Data', fontsize=12, fontweight='bold')
ax.set_ylabel('Ponto de Interesse', fontsize=12, fontweight='bold')

# Rotacionar labels do eixo x
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()

# Salvar figura
arquivo_heatmap = caminho_output / f"heatmap_diferencas_{data_rodada}.png"
plt.savefig(arquivo_heatmap, dpi=300, bbox_inches='tight')
print(f"\nHeatmap salvo em: {arquivo_heatmap}")

plt.show()

# Salvar matriz de diferenças
arquivo_matriz = caminho_output / f"matriz_diferencas_{data_rodada}.csv"
matriz_diferencas.to_csv(arquivo_matriz)
print(f"Matriz de diferenças salva em: {arquivo_matriz}")

## 10. Heatmap de Diferenças Absolutas

In [ ]:
# Criar matriz para heatmap de diferenças absolutas
matriz_dif_abs = dados_comparacao.pivot_table(
    index='ponto',
    columns='data',
    values='diferenca_abs',
    aggfunc='mean'
).fillna(0)

# Configurar figura
fig, ax = plt.subplots(figsize=(14, 8))

# Criar heatmap
sns.heatmap(
    matriz_dif_abs,
    cmap='YlOrRd',
    annot=True,
    fmt='.1f',
    linewidths=0.5,
    cbar_kws={'label': 'Diferença Absoluta (mm)'},
    ax=ax
)

# Formatação
ax.set_title(f'Heatmap de Diferenças Absolutas (|ONS - TOK|)\n{nome_modelo} - {data_rodada}', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Data', fontsize=12, fontweight='bold')
ax.set_ylabel('Ponto de Interesse', fontsize=12, fontweight='bold')

# Rotacionar labels do eixo x
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()

# Salvar figura
arquivo_heatmap_abs = caminho_output / f"heatmap_diferencas_absolutas_{data_rodada}.png"
plt.savefig(arquivo_heatmap_abs, dpi=300, bbox_inches='tight')
print(f"\nHeatmap de diferenças absolutas salvo em: {arquivo_heatmap_abs}")

plt.show()

## 11. Visualizações Adicionais

In [ ]:
# Gráfico de barras: Acumulado de diferenças por ponto
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Diferenças positivas e negativas
acumulados_plot = acumulados[['Acumulado_Positivo_mm', 'Acumulado_Negativo_mm']].plot(
    kind='bar',
    ax=ax1,
    color=['green', 'red'],
    alpha=0.7
)
ax1.set_title(f'Acumulado de Diferenças por Ponto\n{nome_modelo} - {data_rodada}',
              fontsize=12, fontweight='bold')
ax1.set_xlabel('Ponto de Interesse', fontsize=10, fontweight='bold')
ax1.set_ylabel('Precipitação Acumulada (mm)', fontsize=10, fontweight='bold')
ax1.legend(['Positivas (ONS > TOK)', 'Negativas (ONS < TOK)'])
ax1.grid(axis='y', alpha=0.3)
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')

# Gráfico 2: Desvio padrão por ponto
tabela_desvios.plot(kind='bar', ax=ax2, color='orange', alpha=0.7, legend=False)
ax2.set_title(f'Desvio Padrão das Diferenças por Ponto\n{nome_modelo} - {data_rodada}',
              fontsize=12, fontweight='bold')
ax2.set_xlabel('Ponto de Interesse', fontsize=10, fontweight='bold')
ax2.set_ylabel('Desvio Padrão (mm)', fontsize=10, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()

# Salvar figura
arquivo_barras = caminho_output / f"graficos_acumulados_{data_rodada}.png"
plt.savefig(arquivo_barras, dpi=300, bbox_inches='tight')
print(f"\nGráficos de barras salvos em: {arquivo_barras}")

plt.show()

## 12. Distribuição das Diferenças

In [ ]:
# Histograma e boxplot das diferenças
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histograma
ax1.hist(dados_comparacao['diferenca'], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero')
ax1.axvline(x=dados_comparacao['diferenca'].mean(), color='green', linestyle='--', linewidth=2, label='Média')
ax1.set_title(f'Distribuição das Diferenças\n{nome_modelo} - {data_rodada}',
              fontsize=12, fontweight='bold')
ax1.set_xlabel('Diferença (mm)', fontsize=10, fontweight='bold')
ax1.set_ylabel('Frequência', fontsize=10, fontweight='bold')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Boxplot por ponto
dados_comparacao.boxplot(column='diferenca', by='ponto', ax=ax2)
ax2.set_title(f'Boxplot das Diferenças por Ponto\n{nome_modelo} - {data_rodada}',
              fontsize=12, fontweight='bold')
ax2.set_xlabel('Ponto de Interesse', fontsize=10, fontweight='bold')
ax2.set_ylabel('Diferença (mm)', fontsize=10, fontweight='bold')
ax2.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
plt.suptitle('')  # Remove título padrão do boxplot

plt.tight_layout()

# Salvar figura
arquivo_dist = caminho_output / f"distribuicao_diferencas_{data_rodada}.png"
plt.savefig(arquivo_dist, dpi=300, bbox_inches='tight')
print(f"\nGráficos de distribuição salvos em: {arquivo_dist}")

plt.show()

## 13. Relatório Consolidado

In [ ]:
# Criar relatório consolidado
relatorio = f"""
{'='*80}
RELATÓRIO DE COMPARAÇÃO DE PRECIPITAÇÃO - ONS vs TOK
{'='*80}

Modelo: {nome_modelo}
Horizonte: {horizonte}
Data da Rodada: {data_rodada}

{'='*80}
ESTATÍSTICAS GERAIS
{'='*80}

Total de registros comparados: {dados_comparacao.shape[0]}
Número de pontos de interesse: {dados_comparacao['ponto'].nunique()}
Período analisado: {dados_comparacao['data'].min()} a {dados_comparacao['data'].max()}

Precipitação Total ONS: {dados_comparacao['precipitacao_mm_ons'].sum():.2f} mm
Precipitação Total TOK: {dados_comparacao['precipitacao_mm_tok'].sum():.2f} mm

{'='*80}
ANÁLISE DAS DIFERENÇAS
{'='*80}

Diferença Média: {dados_comparacao['diferenca'].mean():.2f} mm
Desvio Padrão das Diferenças: {dados_comparacao['diferenca'].std():.2f} mm
Diferença Mínima: {dados_comparacao['diferenca'].min():.2f} mm
Diferença Máxima: {dados_comparacao['diferenca'].max():.2f} mm

Acumulado de Diferenças Positivas: {acumulado_positivo.sum():.2f} mm
Acumulado de Diferenças Negativas: {acumulado_negativo.sum():.2f} mm
Diferença Líquida Total: {acumulados['Total_Liquido_mm'].sum():.2f} mm

Percentual de registros com ONS > TOK: {(dados_comparacao['diferenca'] > 0).sum() / len(dados_comparacao) * 100:.1f}%
Percentual de registros com ONS < TOK: {(dados_comparacao['diferenca'] < 0).sum() / len(dados_comparacao) * 100:.1f}%
Percentual de registros com ONS = TOK: {(dados_comparacao['diferenca'] == 0).sum() / len(dados_comparacao) * 100:.1f}%

{'='*80}
ARQUIVOS GERADOS
{'='*80}

Diretório de saída: {caminho_output}

1. estatisticas_por_ponto_{data_rodada}.csv
2. acumulados_diferencas_{data_rodada}.csv
3. desvio_padrao_diferencas_{data_rodada}.csv
4. matriz_diferencas_{data_rodada}.csv
5. heatmap_diferencas_{data_rodada}.png
6. heatmap_diferencas_absolutas_{data_rodada}.png
7. graficos_acumulados_{data_rodada}.png
8. distribuicao_diferencas_{data_rodada}.png
9. relatorio_consolidado_{data_rodada}.txt

{'='*80}
"""

print(relatorio)

# Salvar relatório
arquivo_relatorio = caminho_output / f"relatorio_consolidado_{data_rodada}.txt"
with open(arquivo_relatorio, 'w', encoding='utf-8') as f:
    f.write(relatorio)

print(f"\nRelatório consolidado salvo em: {arquivo_relatorio}")

## 14. Resumo Final

Este notebook realizou a comparação completa entre os dados de precipitação das fontes ONS e TOK.

### Principais Outputs:

1. **Tabelas com Heatmaps**: Visualizações das diferenças por ponto de interesse e data
2. **Acumulados**: Diferenças positivas e negativas acumuladas por ponto
3. **Desvio Padrão**: Análise da variabilidade das diferenças
4. **Relatório Consolidado**: Documento com todas as estatísticas principais

### Próximos Passos:

- Ajuste os parâmetros na seção 1 conforme necessário
- Adapte as funções de carregamento de dados se o formato dos seus arquivos for diferente
- Execute todas as células para gerar uma nova análise
- Os outputs serão organizados no diretório de saída conforme a estrutura de input